In [1]:
import os
gpu_device = 2    # número identificador del device puede ser: 0, 1, 2, o 3
os.environ["CUDA_VISIBLE_DEVICES"]=str(gpu_device)
from numba import cuda
cuda.select_device(0)

<weakproxy at 0x7fc2383086d0; to 'numba.cuda.cudadrv.driver.Device' at 0x7fc2384b5d30>

In [2]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
import torch
import pandas as pd
import random
import math
from sklearn.metrics import classification_report, accuracy_score
from collections import defaultdict

/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
model_name="dmis-lab/biobert-base-cased-v1.1"

In [4]:
%run ../utils/data.ipynb
%run ../utils/BERT-mix.ipynb

['biopsy report', 'nodal staging ultrasound report', 'normal control or revision report', 'mammography and ultrasound', 'only mammography study', 'only ultrasound study', 'first degree', 'no family history', 'second degree', 'no prosthesis', 'yes prosthesis', 'BI-RADS 0', 'BI-RADS 1', 'BI-RADS 2', 'BI-RADS 3', 'BI-RADS 4A', 'BI-RADS 4B', 'BI-RADS 4C', 'BI-RADS 5', 'BI-RADS 6', 'ACR A', 'ACR B', 'ACR C', 'ACR D', 'unknown density mammo', 'no calcifications', 'yes calcifications', 'no ganglio', 'yes ganglio', 'fibroglandular and fat', 'heterogeneous fibroglandular', 'homogeneous fatty', 'homogeneous fibroglandular', 'unknown density echo', 'no lymph benign', 'yes lymph benign', 'no lymph suspicious', 'yes lymph suspicious', 'no cyst', 'yes cyst', 'no ectasia', 'yes ectasia', 'no nodules', 'yes nodules', 'irregular', 'lobulated', 'oval', 'round', 'unknown shape', 'circumscribed', 'indistinct', 'not circumscribed', 'spiculated', 'unknown margin', 'complex cystic and solid', 'heterogeneous'

In [5]:
training_args = TrainingArguments(
    output_dir='./results',          # Carpeta para guardar el modelo
    num_train_epochs=5,             # Número de épocas
    per_device_train_batch_size=16,  # Tamaño del batch
    per_device_eval_batch_size=16,   # Tamaño del batch de validación
    learning_rate=0.01, 
    weight_decay=0.05,              # Decaimiento del peso
    logging_dir='./logs',           # Carpeta para los logs
    logging_steps=10,
    evaluation_strategy="epoch",    # Evaluar durante el entrenamiento
    save_strategy="epoch"           # Guardar el modelo al final de cada época
)

training_args_all= TrainingArguments(
    output_dir='./results',          # Carpeta para guardar el modelo
    num_train_epochs=5,             # Número de épocas
    per_device_train_batch_size=16,  # Tamaño del batch
    per_device_eval_batch_size=16,   # Tamaño del batch de validación
    learning_rate=0.00005, 
    weight_decay=0.05,              # Decaimiento del peso
    logging_dir='./logs',           # Carpeta para los logs
    logging_steps=10,
    evaluation_strategy="epoch",    # Evaluar durante el entrenamiento
    save_strategy="epoch"           # Guardar el modelo al final de cada época
)

/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [7]:
len(outputs)

64

In [6]:
predicted = np.load("predicted_bio.npy")
tested = np.load("tested_bio.npy")
evaluate_per_question(predicted, tested, DICTIONARY)


🔍 Per-question classification reports:


📘 Question: birads
              precision    recall  f1-score   support

   BI-RADS 0       0.67      0.67      0.67         3
   BI-RADS 1       0.96      1.00      0.98        24
   BI-RADS 2       1.00      1.00      1.00       107
   BI-RADS 3       0.95      1.00      0.98        20
  BI-RADS 4A       0.81      0.95      0.88        22
  BI-RADS 4B       0.50      0.25      0.33         4
  BI-RADS 4C       1.00      0.33      0.50         3
   BI-RADS 5       1.00      0.50      0.67         2
   BI-RADS 6       0.00      0.00      0.00         1

    accuracy                           0.95       186
   macro avg       0.77      0.63      0.67       186
weighted avg       0.95      0.95      0.94       186


📘 Question: calcifications_benign
                    precision    recall  f1-score   support

 no calcifications       0.98      0.99      0.98        99
yes calcifications       0.99      0.98      0.98        87

          accurac

/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(resu

In [8]:
predicted, tested=cross_validation(dataset_final,targets)

EPOCH  0
                                                                                             text
529-854-912-20231002-160122_family              Question: does the patient have any family his...
420-237-263-20221121-115155_density_echo        Question: what is the breast density found in ...
192-976-987-20230612-125013.437_ductal_ectasia  Question: does the following breast medical re...
278-662-412-20211220-123944_density_echo        Question: what is the breast density found in ...
519-750-939-20220627-113153.437_tipo            Question: is the following breast medical repo...
...                                                                                           ...
278-662-412-20211220-123944_lymph_benign        Question: does the following breast medical re...
261-477-582-20230602-103817_family              Question: does the patient have any family his...
346-239-757-20230622-171503_nodules_margin      Question: what is the margin of the first nodu...
136-563-658

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at dmis-lab/biobert-base-cased-v1.1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Map:   0%|          | 0/2724 [00:00<?, ? examples/s]

⚠️ Truncation occurred!
Original length: 536, Truncated to: 512
Sample text: Question: does the following breast medical report mention any suspicious axillary lymph nodes in the ultrasound exam? Extra information: if a lymph node is suspicious the report will recomend a biops ...

⚠️ Truncation occurred!
Original length: 545, Truncated to: 512
Sample text: Question: what diagnostic technique was used in the following breast medical report? Extra information: biopsy reports, simple cysts and analysis of lymph or axillary nodes are only seen on ultrasound ...

⚠️ Truncation occurred!
Original length: 528, Truncated to: 512
Sample text: Question: is the following breast medical report a biopsy report or a nodal staging ultrasound report? Extra information: biopsy reports are Image-Guided Biopsy or Fine needle aspiration and is normal ...

⚠️ Truncation occurred!
Original length: 535, Truncated to: 512
Sample text: Question: what is the margin of the first nodule described in the ultrasou

Map:  37%|███▋      | 1000/2724 [00:01<00:03, 573.32 examples/s]

⚠️ Truncation occurred!
Original length: 574, Truncated to: 512
Sample text: Question: what diagnostic technique was used in the following breast medical report? Extra information: biopsy reports, simple cysts and analysis of lymph or axillary nodes are only seen on ultrasound ...

⚠️ Truncation occurred!
Original length: 541, Truncated to: 512
Sample text: Question: does the following breast medical report mention any suspicious axillary lymph nodes in the ultrasound exam? Extra information: if a lymph node is suspicious the report will recomend a biops ...

⚠️ Truncation occurred!
Original length: 514, Truncated to: 512
Sample text: Question: what is the breast density found in the mammography study of the following breast medical report? Extra information: breast density in mammography is classified into four categories: ACR A ( ...

⚠️ Truncation occurred!
Original length: 530, Truncated to: 512
Sample text: Question: what is the margin of the first nodule described in the ultrasou

...

Epoch,Training Loss,Validation Loss
1,0.343200,0.508031
2,0.208500,0.307243
3,0.088100,0.351035
4,0.052000,0.411973
5,0.098800,0.365985


[40 43 56 60 53 48 63  5  0 13 26 32 21 40  6 27 34 36 42  9 38  3  2 13
 26 29 21 41  7 27 34 36 42  9 39  3  2 13 26 32 21 40  8 27 34 36 42  9
 38  3  2 15 25 33 22 40  7 27 34 36 43  9 39  3  2 12 25 33 21 40  7 27
 34 36 43 58 59 53 48  9 38  3  2 13 25 30 23 40  7 27 34 36 42  9 39  3
  2 15 25 32 24 40  7 27 34 36 43  9 38  5  2 13 26 32 21 40  7 28 34 36
 42  9 38  3  2 14 26 30 23 40  7 27 34 36 42  9 39  3  2 13 25 30 22 40
  7 27 34 36 43 55 60 49 46 63  9 38  3  2 13 25 32 20 41  7 27 34 36 43
 56 60 49 46 63  9 38  3  2 16 26 32 22 40  7 27 35 36 43 58 59 50 47  9
 38  3  2 13 25 30 22 40  7 27 34 36 43 56 60 49 45 63  9 39  3  2 14 25
 30 23 40  7 27 34 36 43 56 60 49 46 63  9 39  3  2 15 25 33 24 40  6 27
 34 36 43 58 59 49 48  9 38  5  2 14 26 30 22 40  6 27 34 36 43 56 60 49
 46 63  9 38  3  2  5  2 12 25 32 21 40  7 27 35 36 42  9 38  3  2 13 26
 32 21 40  6 27 34 36 42  9 38  3  2]
Accuracy: 0.95
Classification Report:
              precision    recall  f1-score   su

/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/ho

                precision    recall  f1-score   support

 no prosthesis       0.99      1.00      1.00       176
yes prosthesis       1.00      0.90      0.95        10

      accuracy                           0.99       186
     macro avg       1.00      0.95      0.97       186
  weighted avg       0.99      0.99      0.99       186


📘 Question: simple_cyst
              precision    recall  f1-score   support

     no cyst       0.99      0.98      0.99       101
    yes cyst       0.98      0.99      0.98        85

    accuracy                           0.98       186
   macro avg       0.98      0.98      0.98       186
weighted avg       0.98      0.98      0.98       186


📘 Question: tecnica
                            precision    recall  f1-score   support

mammography and ultrasound       0.99      0.99      0.99       150
    only mammography study       0.00      0.00      0.00         2
     only ultrasound study       0.98      1.00      0.99        60

              

/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(resu

In [1]:
np.save("predicted_bio.npy", predicted)
np.save("tested_bio.npy", tested)

NameError: name 'np' is not defined

In [ ]:
✅ Accuracy for: tipo
  biopsy report: 0.8000
  nodal staging ultrasound report: 0.0000
  normal control or revision report: 1.0000
🎯 Overall accuracy: 0.9528

✅ Accuracy for: tecnica
  mammography and ultrasound: 0.9867
  only mammography study: 0.0000
  only ultrasound study: 1.0000
🎯 Overall accuracy: 0.9811

✅ Accuracy for: family
  first degree: 0.8889
  no family history: 0.9937
  second degree: 0.7778
🎯 Overall accuracy: 0.9731

✅ Accuracy for: prosthesis
  no prosthesis: 1.0000
  yes prosthesis: 0.9000
🎯 Overall accuracy: 0.9946

✅ Accuracy for: birads
  BI-RADS 0: 0.0000
  BI-RADS 1: 1.0000
  BI-RADS 2: 1.0000
  BI-RADS 3: 1.0000
  BI-RADS 4A: 1.0000
  BI-RADS 4B: 0.5000
  BI-RADS 4C: 0.0000
  BI-RADS 5: 0.0000
  BI-RADS 6: 0.0000
🎯 Overall accuracy: 0.9409

✅ Accuracy for: density_mammo
  ACR A: 1.0000
  ACR B: 0.9512
  ACR C: 0.9872
  ACR D: 0.9286
  unknown density mammo: 0.9722
🎯 Overall accuracy: 0.9677

✅ Accuracy for: calcifications_benign
  no calcifications: 1.0000
  yes calcifications: 0.9770
🎯 Overall accuracy: 0.9892

✅ Accuracy for: ganglio_mamo
  no ganglio: 0.9944
  yes ganglio: 0.4286
🎯 Overall accuracy: 0.9731

✅ Accuracy for: density_echo
  fibroglandular and fat: 0.8333
  heterogeneous fibroglandular: 0.9750
  homogeneous fatty: 0.2500
  homogeneous fibroglandular: 0.9649
  unknown density echo: 0.9091
🎯 Overall accuracy: 0.9355

✅ Accuracy for: lymph_benign
  no lymph benign: 0.9673
  yes lymph benign: 0.8485
🎯 Overall accuracy: 0.9462

✅ Accuracy for: lymph_suspicious
  no lymph suspicious: 0.9942
  yes lymph suspicious: 0.6667
🎯 Overall accuracy: 0.9677

✅ Accuracy for: simple_cyst
  no cyst: 0.9703
  yes cyst: 0.9882
🎯 Overall accuracy: 0.9785

✅ Accuracy for: ductal_ectasia
  no ectasia: 1.0000
  yes ectasia: 0.9375
🎯 Overall accuracy: 0.9946

✅ Accuracy for: nodules_echo
  no nodules: 0.9231
  yes nodules: 0.9512
🎯 Overall accuracy: 0.9355

✅ Accuracy for: nodules_shape
  irregular: 0.0000
  lobulated: 0.8462
  oval: 0.9600
  round: 0.0000
  unknown shape: 0.9250
🎯 Overall accuracy: 0.8780

✅ Accuracy for: nodules_margin
  circumscribed: 0.9394
  indistinct: 0.3333
  not circumscribed: 0.0000
  spiculated: 0.0000
  unknown margin: 0.9524
🎯 Overall accuracy: 0.8780

✅ Accuracy for: nodules_echogenicity
  complex cystic and solid: 0.0000
  heterogeneous: 0.0000
  hypoechoic: 1.0000
  isoechoic: 0.0000
  unknown echogenicity: 0.9310
🎯 Overall accuracy: 0.9146

✅ Accuracy for: nodules_known
  no known: 0.8421
  yes known: 0.8182
🎯 Overall accuracy: 0.8293

✅ Accuracy for: nodules_stable
  grown stable: 0.0000
  shrunk stable: 0.0000
  unknown stable: 0.0000
  yes stable: 0.9429
🎯 Overall accuracy: 0.7500

In [ ]:
✅ Accuracy for: tipo
  biopsy report: 0.8500
  nodal staging ultrasound report: 0.3333
  normal control or revision report: 1.0000
🎯 Overall accuracy: 0.9670

✅ Accuracy for: tecnica
  mammography and ultrasound: 0.9933
  only mammography study: 0.0000
  only ultrasound study: 0.9833
🎯 Overall accuracy: 0.9811

✅ Accuracy for: family
  first degree: 0.9444
  no family history: 1.0000
  second degree: 0.8889
🎯 Overall accuracy: 0.9892

✅ Accuracy for: prosthesis
  no prosthesis: 1.0000
  yes prosthesis: 0.9000
🎯 Overall accuracy: 0.9946

✅ Accuracy for: birads
  BI-RADS 0: 0.6667
  BI-RADS 1: 1.0000
  BI-RADS 2: 1.0000
  BI-RADS 3: 1.0000
  BI-RADS 4A: 0.9545
  BI-RADS 4B: 1.0000
  BI-RADS 4C: 0.3333
  BI-RADS 5: 0.0000
  BI-RADS 6: 0.0000
  unknown BI-RADS: [No samples]
🎯 Overall accuracy: 0.9624

✅ Accuracy for: density_mammo
  ACR A: 0.6667
  ACR B: 0.9512
  ACR C: 0.9872
  ACR D: 0.9643
  unknown density mammo: 0.9722
🎯 Overall accuracy: 0.9677

✅ Accuracy for: calcifications_benign
  no calcifications: 0.9899
  yes calcifications: 0.9655
🎯 Overall accuracy: 0.9785

✅ Accuracy for: ganglio_mamo
  no ganglio: 0.9944
  yes ganglio: 0.8571
🎯 Overall accuracy: 0.9892

✅ Accuracy for: density_echo
  fibroglandular and fat: 0.9167
  heterogeneous fibroglandular: 0.9750
  homogeneous fatty: 0.5000
  homogeneous fibroglandular: 0.9825
  unknown density echo: 0.8788
🎯 Overall accuracy: 0.9462

✅ Accuracy for: lymph_benign
  no lymph benign: 0.9804
  yes lymph benign: 0.7879
🎯 Overall accuracy: 0.9462

✅ Accuracy for: lymph_suspicious
  no lymph suspicious: 0.9942
  yes lymph suspicious: 0.6667
🎯 Overall accuracy: 0.9677

✅ Accuracy for: simple_cyst
  no cyst: 0.9802
  yes cyst: 0.9882
🎯 Overall accuracy: 0.9839

✅ Accuracy for: ductal_ectasia
  no ectasia: 1.0000
  yes ectasia: 0.9375
🎯 Overall accuracy: 0.9946

In [12]:
evaluate_per_question(predicted, tested, DICTIONARY)


🔍 Per-question classification reports:


📘 Question: birads
                 precision    recall  f1-score   support

      BI-RADS 0       0.00      0.00      0.00         3
      BI-RADS 1       0.96      1.00      0.98        24
      BI-RADS 2       0.90      0.94      0.92       107
      BI-RADS 3       0.65      0.55      0.59        20
     BI-RADS 4A       0.45      0.64      0.53        22
     BI-RADS 4B       0.00      0.00      0.00         4
     BI-RADS 4C       0.00      0.00      0.00         3
      BI-RADS 5       0.00      0.00      0.00         2
      BI-RADS 6       0.00      0.00      0.00         1
unknown BI-RADS       0.00      0.00      0.00         0

      micro avg       0.81      0.81      0.81       186
      macro avg       0.30      0.31      0.30       186
   weighted avg       0.77      0.81      0.78       186


📘 Question: calcifications_benign
                    precision    recall  f1-score   support

 no calcifications       0.94      0.99   

/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(resul

              precision    recall  f1-score   support

  no ectasia       0.99      1.00      0.99       170
 yes ectasia       1.00      0.88      0.93        16

    accuracy                           0.99       186
   macro avg       0.99      0.94      0.96       186
weighted avg       0.99      0.99      0.99       186


📘 Question: family
                   precision    recall  f1-score   support

     first degree       0.94      0.94      0.94        18
no family history       1.00      1.00      1.00       159
    second degree       0.89      0.89      0.89         9

         accuracy                           0.99       186
        macro avg       0.94      0.94      0.94       186
     weighted avg       0.99      0.99      0.99       186


📘 Question: ganglio_mamo
              precision    recall  f1-score   support

  no ganglio       0.99      0.99      0.99       179
 yes ganglio       0.83      0.71      0.77         7

    accuracy                           0.98    

/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/mcarrilero/pytorch_env/lib/python3.13/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(resu